<a href="https://colab.research.google.com/github/KovryakRuslan312005/MH-Kovryak-Ruslan-3-16/blob/main/%D0%9A%D0%BE%D0%B2%D1%80%D1%8F%D0%BA%20%D0%A0%D1%83%D1%81%D0%BB%D0%B0%D0%BD%20%D0%9B%D0%B0%D0%B1%D0%B0%20%E2%84%968%20.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Ковряк Руслан Павлович Лаба №8

1 КРОК - встановлюємо бібліотеки

In [ ]:
!pip install numpy==1.24.4 pandas scikit-learn
!pip install scikit-surprise --no-binary scikit-surprise


2 КРОК - завантажуємо датасет  MovieLens 100k

In [ ]:
import pandas as pd

url = "http://files.grouplens.org/datasets/movielens/ml-100k/u.data"
df = pd.read_csv(url, sep='\t', names=['user_id', 'movie_id', 'rating', 'timestamp'])
df = df[['user_id', 'movie_id', 'rating']]


3 КРОК - Використовуємо метод найменших квадратів

In [ ]:
 from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split

X = df[['user_id', 'movie_id']]
y = df['rating']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

model_lr = LinearRegression().fit(X_train, y_train)

user_id = 10
movies_not_rated = df[~df['movie_id'].isin(df[df['user_id'] == user_id]['movie_id'])]['movie_id'].unique()
input_data = pd.DataFrame({'user_id': user_id, 'movie_id': movies_not_rated})
input_data['predicted_rating'] = model_lr.predict(input_data)

print(input_data.sort_values(by='predicted_rating', ascending=False).head())


     user_id  movie_id  predicted_rating
114       10         2          3.812466
182       10         3          3.811823
59        10         5          3.810537
881       10         6          3.809893
296       10         8          3.808607


4 КРОК - Спуск градієнта

In [ ]:
import numpy as np

n_users = df['user_id'].nunique()
n_movies = df['movie_id'].nunique()
R = np.zeros((n_users, n_movies))
for row in df.itertuples():
    R[row.user_id - 1, row.movie_id - 1] = row.rating

n_factors = 10
P = np.random.rand(n_users, n_factors)
Q = np.random.rand(n_movies, n_factors)
alpha = 0.005
epochs = 20

for epoch in range(epochs):
    for i in range(n_users):
        for j in range(n_movies):
            if R[i][j] > 0:
                eij = R[i][j] - np.dot(P[i, :], Q[j, :])
                P[i, :] += alpha * (eij * Q[j, :])
                Q[j, :] += alpha * (eij * P[i, :])

pred_matrix = np.dot(P, Q.T)
user_ratings = pred_matrix[user_id - 1]
top_movies = np.argsort(user_ratings)[::-1][:5]

print("Рекомендовані фільми (ID):", top_movies + 1)


Рекомендовані фільми (ID): [ 169 1367  272  318   64]


5 КРОК - Кластеризація користувачів

In [ ]:
from sklearn.cluster import KMeans

user_movie_matrix = df.pivot_table(index='user_id', columns='movie_id', values='rating').fillna(0)
kmeans = KMeans(n_clusters=5, random_state=0)
clusters = kmeans.fit_predict(user_movie_matrix)

user_movie_matrix['cluster'] = clusters
user_cluster = user_movie_matrix.loc[user_id]['cluster']
cluster_users = user_movie_matrix[user_movie_matrix['cluster'] == user_cluster].drop('cluster', axis=1)
mean_ratings = cluster_users.mean().sort_values(ascending=False)

print("Топ-5 фільмів у кластері користувача:\n", mean_ratings.head())


Топ-5 фільмів у кластері користувача:
 movie_id
483    4.287879
134    4.196970
50     4.106061
174    4.060606
100    4.030303
dtype: float64


Висновк- В цій лабораторній роботі ми працювали з датасетом MovieLens 100k.Робили з цим датасетом:Метод найменших квадратів (лінійна регресія),Градієнтний спуск для матричної факторизації,Кластеризація (KMeans).